# Customize output format in prompt

## Load model

In [38]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-search-preview")

model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0xffff60acfb00>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0xffff60af45c0>, root_client=<openai.OpenAI object at 0xffff60b47d10>, root_async_client=<openai.AsyncOpenAI object at 0xffff60acf560>, model_name='gpt-4o-search-preview', model_kwargs={}, openai_api_key=SecretStr('**********'))

## Load prompt

In [39]:
from langchain_core.prompts import load_prompt
prompt = load_prompt(path='../prompts/single-day/ES.json')

prompt

PromptTemplate(input_variables=['DATE', 'FREQUENCY', 'RETURN', 'TICKER'], input_types={}, partial_variables={}, template='El día {DATE}, el activo financiero con ticker {TICKER} registró un retorno de {RETURN}%, calculado como la variación porcentual entre el precio de cierre de esa fecha y el de la fecha anterior correspondiente según la frecuencia de cálculo definida: {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nExplica los motivos que podrían justificar este movimiento extremo, utilizando noticias relevantes publicadas en torno a esa fecha. Proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.')

## Custom Output Parser

### Create Pydantic model

`/workspace/modules/models.py`

### Associate Pydantic model with parser

In [47]:
from langchain_core.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=News)

parser

PydanticOutputParser(pydantic_object=<class '__main__.News'>)

### Add parser to prompt

In [48]:
prompt.template = 'Answer the QUERY in Spanish and following the format: {format_instructions}\n\nQUERY:\n\n' + prompt.template
prompt.template 

'Answer the QUERY in Spanish and following the format: {format_instructions}\n\nQUERY:\n\n{format_instructions}\n\nEl día {DATE}, el activo financiero con ticker {TICKER} registró un retorno de {RETURN}%, calculado como la variación porcentual entre el precio de cierre de esa fecha y el de la fecha anterior correspondiente según la frecuencia de cálculo definida: {FREQUENCY} (por ejemplo: diaria D, mensual M, trimestral Q o anual Y).\n\nExplica los motivos que podrían justificar este movimiento extremo, utilizando noticias relevantes publicadas en torno a esa fecha. Proporciona una explicación fundamentada, la fecha de publicación de la noticia y un enlace a la fuente.'

In [49]:
prompt.partial_variables.update({"format_instructions": parser.get_format_instructions()})
prompt

PromptTemplate(input_variables=['DATE', 'FREQUENCY', 'RETURN', 'TICKER'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"stock": {"description": "símbolo de la acción", "title": "Stock", "type": "string"}, "date": {"description": "fecha de la noticia", "title": "Date", "type": "string"}, "value": {"description": "valor acumulado significativo de la acción", "title": "Value", "type": "number"}, "title": {"description": "título de la noticia", "title": "Title", "type": "string"}, "url": {"description": "url de

## Chain

### Define chain

In [50]:
chain = prompt | model | parser
chain

PromptTemplate(input_variables=['DATE', 'FREQUENCY', 'RETURN', 'TICKER'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"stock": {"description": "símbolo de la acción", "title": "Stock", "type": "string"}, "date": {"description": "fecha de la noticia", "title": "Date", "type": "string"}, "value": {"description": "valor acumulado significativo de la acción", "title": "Value", "type": "number"}, "title": {"description": "título de la noticia", "title": "Title", "type": "string"}, "url": {"description": "url de

### Preprocess input data

In [51]:
ticker = 'AAPL'
freq = 'D'

In [52]:
import utils

r = utils.calculate_significant_returns_simple(
    ticker=ticker,
    start='2024-01-01',
    end='2024-12-31',
    freq=freq,
    n=1
)

r

[*********************100%***********************]  1 of 1 completed


Date
2024-06-11    7.26492
Name: AAPL, dtype: float64

In [53]:
date = r.index[0].strftime('%Y-%m-%d')
change = round(r.values[0], 2)

date, change

('2024-06-11', 7.26)

### Invoke chain

In [54]:
output = chain.invoke({
    'DATE': date,
    'RETURN': change,
    'TICKER': ticker,
    'FREQUENCY': freq
})

print(output)

stock='AAPL' date='2024-06-11' value=7.26 title='Las acciones de Apple suben a un nivel récord tras los anuncios de IA' url='https://eltiempolatino.com/2024/06/11/economia/acciones-de-apple-suben-a-un-nivel-record/' source='El Tiempo Latino' explanation="El 10 de junio de 2024, durante la Conferencia Mundial de Desarrolladores (WWDC), Apple presentó 'Apple Intelligence', su plataforma de inteligencia artificial generativa. Este anuncio generó un optimismo significativo en Wall Street, impulsando las acciones de Apple a un cierre récord el 11 de junio de 2024, con un aumento del 7%. Los analistas elogiaron la integración de la IA en el ecosistema de Apple, anticipando un nuevo ciclo de actualización de productos y un crecimiento acelerado de los ingresos."


In [55]:
data = output.model_dump()
data

{'stock': 'AAPL',
 'date': '2024-06-11',
 'value': 7.26,
 'title': 'Las acciones de Apple suben a un nivel récord tras los anuncios de IA',
 'url': 'https://eltiempolatino.com/2024/06/11/economia/acciones-de-apple-suben-a-un-nivel-record/',
 'source': 'El Tiempo Latino',
 'explanation': "El 10 de junio de 2024, durante la Conferencia Mundial de Desarrolladores (WWDC), Apple presentó 'Apple Intelligence', su plataforma de inteligencia artificial generativa. Este anuncio generó un optimismo significativo en Wall Street, impulsando las acciones de Apple a un cierre récord el 11 de junio de 2024, con un aumento del 7%. Los analistas elogiaron la integración de la IA en el ecosistema de Apple, anticipando un nuevo ciclo de actualización de productos y un crecimiento acelerado de los ingresos."}

In [56]:
import pandas as pd
pd.DataFrame([data]).style

,stock,date,value,title,url,source,explanation
0,AAPL,2024-06-11,7.260000,Las acciones de Apple suben a un nivel récord tras los anuncios de IA,https://eltiempolatino.com/2024/06/11/economia/acciones-de-apple-suben-a-un-nivel-record/,El Tiempo Latino,"El 10 de junio de 2024, durante la Conferencia Mundial de Desarrolladores (WWDC), Apple presentó 'Apple Intelligence', su plataforma de inteligencia artificial generativa. Este anuncio generó un optimismo significativo en Wall Street, impulsando las acciones de Apple a un cierre récord el 11 de junio de 2024, con un aumento del 7%. Los analistas elogiaron la integración de la IA en el ecosistema de Apple, anticipando un nuevo ciclo de actualización de productos y un crecimiento acelerado de los ingresos."


## Multiple days

In [57]:
import utils

r = utils.calculate_significant_returns_simple(
    ticker=ticker,
    start='2024-01-01',
    end='2024-12-31',
    freq=freq,
    n=5
)

r

[*********************100%***********************]  1 of 1 completed

Date
2024-06-11    7.264920
2024-05-03    5.981614
2024-08-05   -4.816705
2024-04-11    4.327102
2024-03-21   -4.085741
Name: AAPL, dtype: float64

In [61]:
results = []

for date, change in r.items():
    try:
        output = chain.invoke({
            'DATE': date.strftime('%Y-%m-%d'),
            'RETURN': round(change, 2),
            'TICKER': ticker,
            'FREQUENCY': freq
        })
        results.append(output.model_dump())
    except Exception as e:
        print(f"Error: {e}")
        continue

In [63]:
df = pd.DataFrame([r for r in results])
df.style

,stock,date,value,title,url,source,explanation
0,AAPL,2024-06-11,7.260000,Apple presenta iOS 18 y Apple Intelligence en la WWDC 2024,https://www.apple.com/es/newsroom/2024/03/apples-worldwide-developers-conference-returns-june-10-2024/,Apple Newsroom,"El anuncio de iOS 18 y Apple Intelligence generó entusiasmo en el mercado, reflejándose en un aumento del 7.26% en el precio de las acciones de Apple."
1,AAPL,2024-05-02,5.980000,Apple informa los resultados del segundo trimestre,https://www.apple.com/us-es/newsroom/2024/05/apple-reports-second-quarter-results/,Apple Newsroom,"El informe financiero del segundo trimestre fiscal de 2024 de Apple mostró ingresos de $90,800 millones y ganancias por acción de $1.53, además de un récord en ingresos por servicios y un programa de recompra de acciones por $110,000 millones, lo que generó confianza entre los inversores y contribuyó al aumento en el precio de las acciones."
2,AAPL,2024-08-05,-4.820000,Claves del lunes negro en bolsa: ¿qué significa para la economía mundial?,https://elpais.com/expres/2024-08-06/claves-del-lunes-negro-en-bolsa-que-significa-para-la-economia-mundial.html,El País,"El 5 de agosto de 2024, las acciones de Apple cayeron un 4,82% en el contexto de un 'lunes negro' en los mercados bursátiles globales. Este desplome fue impulsado por el temor a una recesión en Estados Unidos, debido a la creación de solo 114.000 empleos en julio y un aumento en la tasa de desempleo al 4,3%. Además, los resultados mediocres de grandes empresas tecnológicas intensificaron la desconfianza entre los inversores, afectando negativamente a Apple."
3,AAPL,2024-04-11,4.330000,Apple fija como fecha límite el 11 de abril para que los empleados de EE. UU. vuelvan a la oficina,https://noticiasmoviles.com/apple-fija-como-fecha-limite-el-11-de-abril-para-que-los-empleados-de-ee-uu-vuelvan-a-la-oficina/,Noticias Movil,"La decisión de Apple de establecer el 11 de abril como fecha límite para que los empleados corporativos en Estados Unidos regresaran a las oficinas fue interpretada positivamente por los inversores, ya que indica una vuelta a la normalidad operativa y podría mejorar la productividad y colaboración dentro de la empresa."
4,AAPL,2024-03-21,-4.090000,Las acciones de Apple cayeron un 4% después de la demanda de Estados Unidos a la compañía por monopolio,https://www.infobae.com/estados-unidos/2024/03/22/las-acciones-de-apple-cayeron-un-37-despues-de-la-demanda-de-estados-unidos-a-la-compania-por-monopolio/,Infobae,"La caída del 4% en las acciones de Apple el 21 de marzo de 2024 se debió a la demanda presentada por el Departamento de Justicia de Estados Unidos, que acusó a la compañía de prácticas monopolísticas en el mercado de los teléfonos inteligentes. Esta acción legal generó incertidumbre entre los inversores, afectando negativamente el valor de las acciones de Apple."


## Export results

In [65]:
df.to_csv('outputs/news.xlsx', index=False)

In [ ]:
df.to_csv('outputs/news.csv', index=False)